<a href="https://colab.research.google.com/github/srpantano/RAG-1/blob/main/vector_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -qU langchain-community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
file_path = "/content/pdfs/"

Read all the PDFs in the directory

In [ ]:
loader = PyPDFDirectoryLoader(file_path)
docs = loader.load()

Split the documents. This can be useful because some LLMs has a tokens limits.<br/>The number of chunk size and overlap needs to be adjust for performance.<br/>Overlapping context mantains the context.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=100,
  separators=["\n\n", "\n", " ", ""]
)

all_splits = []
for doc in docs:
    splits = text_splitter.split_documents([doc])
    all_splits.extend(splits)

In [ ]:
all_splits[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.0 (Windows)', 'creationdate': '2023-08-07T23:50:52-03:00', 'moddate': '2023-08-07T23:51:30-03:00', 'trapped': '/False', 'source': '/content/pdfs/Unidade3.pdf', 'total_pages': 34, 'page': 0, 'page_label': '1'}, page_content='Conteudista\nProf.ª Dra. Vilma Lima\nRevisão Textual\nNatalia Lotz Mendes\nRevisão Técnica\nProf.ª M.ª Elaine Barreto Batista\nEscolha do Negócio e Definição  \ndo Mercado Alvo')

In [ ]:
%pip install -qU chromadb

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v3",
                                   model_kwargs={"trust_remote_code": True})


In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings,
                                    persist_directory="/content/vectorized_data/chroma_db")

In [ ]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('Groq')

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=1
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

user_question = "Fale-me sobre marcas e patentes no Brasil."

results = vectorstore.similarity_search(query=user_question, k=5)

context = "\n".join([doc.page_content for doc in results])

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente que responde perguntas sobre {subject}.\n\n"
            "Use APENAS o contexto abaixo para responder. Se a resposta não estiver, "
            "no contexto, diga: 'Desculpe, não sei como ajudar com isso.'\n\n"
            "Contexto:\n{context}\n"
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm

response = chain.invoke(
    {
        "subject": "empreendedorismo",
        "context": context,
        "input": user_question,
    }
)

print(response)

content='Em termos gerais, uma marca é um sinal distintivo, visualmente perceptível, que identifica e distingue produtos e serviços de outros análogos, de procedência diversa. Ela é um conceito intangível e complexo, que pode ser definido como uma promessa, uma totalidade de percepções, uma posição diferenciadora nas mentes dos consumidores e um conjunto de atributos, benefícios, convicções e valores que diferenciam e simplificam o processo decisório. No Brasil, as marcas têm prazo de validade de dez anos, contados a partir da data de concessão, podendo ser prorrogado por períodos iguais e sucessivos.\n\nAntes de registrar uma marca no Brasil, é importante realizar uma pesquisa na Junta Comercial e no Cartório para verificar se o nome desejado já está em uso por outra empresa. O registro da marca é feito no Instituto Nacional de Propriedade Industrial (Inpi), e garante ao seu proprietário o direito de exclusividade. Existem diferentes tipos de marca, como marca denominativa, marca figu

In [ ]:
response.content

'Em termos gerais, uma marca é um sinal distintivo, visualmente perceptível, que identifica e distingue produtos e serviços de outros análogos, de procedência diversa. Ela é um conceito intangível e complexo, que pode ser definido como uma promessa, uma totalidade de percepções, uma posição diferenciadora nas mentes dos consumidores e um conjunto de atributos, benefícios, convicções e valores que diferenciam e simplificam o processo decisório. No Brasil, as marcas têm prazo de validade de dez anos, contados a partir da data de concessão, podendo ser prorrogado por períodos iguais e sucessivos.\n\nAntes de registrar uma marca no Brasil, é importante realizar uma pesquisa na Junta Comercial e no Cartório para verificar se o nome desejado já está em uso por outra empresa. O registro da marca é feito no Instituto Nacional de Propriedade Industrial (Inpi), e garante ao seu proprietário o direito de exclusividade. Existem diferentes tipos de marca, como marca denominativa, marca figurativa, 

In [ ]:
response.response_metadata

{'token_usage': {'completion_tokens': 537,
  'prompt_tokens': 1600,
  'total_tokens': 2137,
  'completion_time': 0.83522577,
  'prompt_time': 0.093797953,
  'queue_time': 0.10863059800000001,
  'total_time': 0.929023723},
 'model_name': 'mixtral-8x7b-32768',
 'system_fingerprint': 'fp_c5f20b5bb1',
 'finish_reason': 'stop',
 'logprobs': None}

In [ ]:
response.pretty_print()

================================== Ai Message ==================================

Em termos gerais, uma marca é um sinal distintivo, visualmente perceptível, que identifica e distingue produtos e serviços de outros análogos, de procedência diversa. Ela é um conceito intangível e complexo, que pode ser definido como uma promessa, uma totalidade de percepções, uma posição diferenciadora nas mentes dos consumidores e um conjunto de atributos, benefícios, convicções e valores que diferenciam e simplificam o processo decisório. No Brasil, as marcas têm prazo de validade de dez anos, contados a partir da data de concessão, podendo ser prorrogado por períodos iguais e sucessivos.

Antes de registrar uma marca no Brasil, é importante realizar uma pesquisa na Junta Comercial e no Cartório para verificar se o nome desejado já está em uso por outra empresa. O registro da marca é feito no Instituto Nacional de Propriedade Industrial (Inpi), e garante ao seu proprietário o direito de exclusividade.